首先还是读取我们的数据集，这里我们对数据做一个归一化的处理，方便模型收敛，并且对label做一个one-hot处理，因为是10分类问题，所以num_classes这个参数的值为10。

In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

Using TensorFlow backend.


NameError: name 'keras' is not defined

模型搭建阶段我们依旧采用序贯模型Sequential，整体的结构上文已经提过，这里不再赘述。

In [36]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_141 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_86 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 8, 8, 64)          0         
__________

然后是定义损失函数、优化方法、评估指标。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

模型准备完成后就可以开始训练了，这里我们引入了EarlyStopping，EarlyStopping可以保证模型在准确率不在提升的前提下提前结束训练，其中monitor参数表示的是停止时参考的指标，这里我们采用的是准确率，patience的意思是多少轮指标没有改变模型训练就停止，这里我们使用的是5，也就是说如果训练五轮，准确率依旧没有提升即停止训练。

In [37]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=5, verbose=1)
model.fit(x_train, y_train, batch_size=64, epochs=200, callbacks=[early_stopping])

Epoch 1/200
50000/50000 [==============================] - 19s 378us/step - loss: 1.8772 - acc: 0.2659
Epoch 2/200
50000/50000 [==============================] - 17s 349us/step - loss: 1.4498 - acc: 0.4635
Epoch 3/200
50000/50000 [==============================] - 18s 352us/step - loss: 1.2200 - acc: 0.5600
Epoch 4/200
50000/50000 [==============================] - 18s 350us/step - loss: 1.0829 - acc: 0.6156
Epoch 5/200
50000/50000 [==============================] - 18s 351us/step - loss: 0.9924 - acc: 0.6516
Epoch 6/200
50000/50000 [==============================] - 17s 347us/step - loss: 0.9224 - acc: 0.6782
Epoch 7/200
50000/50000 [==============================] - 17s 348us/step - loss: 0.8775 - acc: 0.6994
Epoch 8/200
50000/50000 [==============================] - 18s 351us/step - loss: 0.8405 - acc: 0.7126
Epoch 9/200
50000/50000 [==============================] - 18s 351us/step - loss: 0.8073 - acc: 0.7237
Epoch 10/200
50000/50000 [==============================] - 17s 348us/ste

最后我们就可以在测试集进行测试了，可以看到最后的准确率在79.28%，当然了，读者也可以自行对上文的VGG参数进行调整来提高模型的最终效果。

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=64)
print('Test accuracy:', score[1])